In [10]:
%run ipy_setup.py

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sqlalchemy

In [4]:
#branded food DB
nutrients = pd.read_table("../data/USDA_BFPD_csv/Nutrients.csv",sep=",")
products = pd.read_table("../data/USDA_BFPD_csv/Products.csv",sep=",")
serving_sizes = pd.read_table("../data/USDA_BFPD_csv/Serving_size.csv",sep=",")
len(list(zip(nutrients["Nutrient_Code"],nutrients["Nutrient_name"])))

import collections
collections.Counter(list(zip(nutrients["Nutrient_Code"],nutrients["Nutrient_name"])))

/Users/ryanneff/anaconda3/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
columns_nut_data=[i.strip() for i in '''ndb_id 
    nutr_id 
    nutr_value 
    num_data_pts 
    std_err 
    src_code 
    deriv_code 
    ref_ndb_no 
    add_nutr_mark 
    num_studies 
    min_val 
    max_val 
    deg_freedom 
    low_eb 
    up_eb 
    stat_cmt 
    addmod_date'''.split("\n")]

columns_food_desc = [i.strip() for i in """ndb_id 
    foodgroup_code 
    long_desc 
    short_desc 
    common_name 
    manufacturer_name 
    survey 
    ref_desc 
    ref_use 
    scientific_name 
    nit_factor 
    prot_factor 
    fat_factor 
    carb_factor""".split("\n")]

columns_nut_def = [i.strip() for i in '''nutr_id 
    units 
    tagname 
    nutr_desc 
    num_decimal_pts 
    sr_order'''.split("\n")]

columns_weight = [i.strip() for i in '''ndb_id 
    seq 
    amount 
    measure_desc 
    gm_weight 
    num_data_pts 
    std_dev'''.split("\n")]

nut_data = pd.read_table("../data/USDA_SR-Leg_csv/NUT_DATA.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_nut_data,encoding='latin-1')

food_desc = pd.read_table("../data/USDA_SR-Leg_csv/FOOD_DES.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_food_desc,encoding='latin-1')

nut_def = pd.read_table("../data/USDA_SR-Leg_csv/NUTR_DEF.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_nut_def,encoding='latin-1')

food_meas = pd.read_table("../data/USDA_SR-Leg_csv/WEIGHT.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_weight,encoding='latin-1')

#nut_data.set_index("NDB_No",inplace=True)
food_desc.set_index("ndb_id",inplace=True)
#food_meas.set_index("NDB_No",inplace=True)

nut_def.set_index("nutr_id",inplace=True)

In [72]:
food_desc[["steak" in i for i in food_desc["long_desc"]]]

,foodgroup_code,long_desc,short_desc,common_name,manufacturer_name,survey,ref_desc,ref_use,scientific_name,nit_factor,prot_factor,fat_factor,carb_factor
ndb_id,,,,,,,,,,,,,
10080,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STEAKS),L...","URMIS #3186, Pork shoulder chop, Shoulder blad...",NaN,Y,"Bone 9%, Connective tissue 15%",24,NaN,6.25,4.27,9.02,3.87
10081,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STKS),LN ...","URMIS #3186, Pork shoulder chop, Shoulder blad...",NaN,Y,"Bone 9%, Connective tissue 17%",26,NaN,6.25,4.27,9.02,3.87
10082,1000,"Pork, fresh, shoulder, blade, boston (steaks),...","PORK,FRSH,SHLDR,BLADE,BOSTON (STEAKS),LN&FAT,C...",NaN,NaN,Y,Bone,23,NaN,6.25,4.27,9.02,3.87
10084,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STEAKS),L...","URMIS # 3186, Pork shoulder chop, Shoulder bla...",NaN,Y,"Bone, 9%, Connective tissue, 15%, Separable fa...",32,NaN,6.25,4.27,9.02,3.87
10085,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STEAKS),L...","URMIS #3186, Pork shoulder chop, Shoulder blad...",NaN,Y,"Bone, 9%, Connective tissue, 17%, Separable fa...",32,NaN,6.25,4.27,9.02,3.87
10086,1000,"Pork, fresh, shoulder, blade, boston (steaks),...","PORK,FRSH,SHLDR,BLADE,BOSTON (STEAKS),LN,CKD,BRLD",NaN,NaN,Y,"Bone 23%, separable fat 10%",33,NaN,6.25,4.27,9.02,3.87
10149,1000,"Pork, cured, ham, steak, boneless, extra lean,...","PORK,CURED,HAM,STEAK,BNLESS,EX LN,UNHTD",NaN,NaN,NaN,NaN,0,NaN,6.25,4.27,9.02,3.87
10945,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BSTN BTT),BLDE (STKS),LN,W/AD...",NaN,NaN,Y,"Bone, 9%, Connective tissue, 19%, Separable fa...",35,NaN,NaN,4.27,9.02,3.87
10946,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BSTN BUTT),BLDE (STKS),LN,W/ ...",NaN,NaN,Y,"Bone, 7%, Connective tissue, 16%, Separable fa...",32,NaN,NaN,NaN,NaN,NaN


In [74]:
food_meas

,ndb_id,seq,amount,measure_desc,gm_weight,num_data_pts,std_dev
0,01001,1,1,"pat (1"" sq, 1/3"" high)",5,NaN,NaN
1,01001,2,1,tbsp,14.20,NaN,NaN
2,01001,3,1,cup,227,NaN,NaN
3,01001,4,1,stick,113,NaN,NaN
4,01002,1,1,"pat (1"" sq, 1/3"" high)",3.80,NaN,NaN
5,01002,2,1,tbsp,9.40,NaN,NaN
6,01002,3,1,cup,151,NaN,NaN
7,01002,4,1,stick,76,NaN,NaN
8,01003,1,1,tbsp,12.80,NaN,NaN
9,01003,2,1,cup,205,NaN,NaN


In [77]:
nut_data

,ndb_id,nutr_id,nutr_value,num_data_pts,std_err,src_code,deriv_code,ref_ndb_no,add_nutr_mark,num_studies,min_val,max_val,deg_freedom,low_eb,up_eb,stat_cmt,addmod_date
0,01001,208,717.,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/01/2010
1,01001,262,0.,0,NaN,7,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/01/2001
2,01001,263,0.,0,NaN,7,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/01/2001
3,01001,268,2999.,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/01/2015
4,01001,301,24.,17,0.,1,A,NaN,NaN,7,19.,30.,4,22.,26.,"2, 3",11/01/2002
5,01001,304,2.,18,0.,1,A,NaN,NaN,7,1.,2.,4,1.,1.,"2, 3",03/01/2003
6,01001,305,24.,17,0.,1,A,NaN,NaN,7,19.,27.,7,22.,24.,"2, 3",03/01/2003
7,01001,306,24.,18,0.,1,A,NaN,NaN,7,17.,28.,2,20.,26.,"2, 3",03/01/2003
8,01001,307,643.,0,NaN,4,NR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/01/2013
9,01001,318,2499.,0,NaN,4,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/01/2010


In [75]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///food.db', echo=False)

In [76]:
nut_data.to_sql("nut_data",con=engine)
nut_def.to_sql("nut_def",con=engine)
food_desc.to_sql("food_desc",con=engine)
food_meas.to_sql("food_meas",con=engine)